In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import string
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer,CountVectorizer
from nltk.stem.porter import PorterStemmer
dataset=pd.read_csv(r'C:\Users\HP\Desktop\ML\Reviews.csv')
dataset=dataset.drop_duplicates(subset={'UserId','Time'},keep='first',inplace=False)
#dataset=dataset.sample(10000)
#dataset.reset_index(inplace = True) 
print(len(dataset))
dataset=dataset.sort_values('Time',ascending=True)




352744


In [2]:
print(dataset['Text'][500])
print(dataset.shape)

I bought these for the office and no one can put them down. I'm ordering more today and it has only been 2 weeks. Didn't have to shop for them either, Amazon brought it to my door!
(352744, 10)


In [3]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer 
import nltk
#nltk.download('wordnet')
  
#lemmatizer = WordNetLemmatizer()

stop=set(stopwords.words('english'))
sno=SnowballStemmer('english')

def clean_html(sentence):
    x=re.compile('<.*?>')
    cleantext=re.sub(x,' ',sentence)
    return cleantext

def clean_punc(sentence):
    cleaned=re.sub(r'[?|!|\'|"|#|.|,|)|(|\|/]',r' ',sentence)
    return cleaned

p_text=[] #processed text
from tqdm import tqdm
for sent in tqdm(dataset['Text'].values,position=0,leave=True):
    sent=clean_html(sent)
    sent=clean_punc(sent)
    s=[]
    for w in sent.split():
        if(w.isalpha() and len(w)>2 and w.lower() not in stop):
            w=(sno.stem(w.lower()))
            #w=lemmatizer.lemmatize(w.lower()) stemming was producing better results
            s.append(w)
    sent=" ".join(s)
    p_text.append(sent)
    
dataset['p_text']=p_text

dataset['p_text'][377]

100%|█████████████████████████████████████████████████████████████████████████| 352744/352744 [08:02<00:00, 731.77it/s]


'sir kensington great job updat classic ketchup wonder product refresh tast ketchup great updat leav disappoint given heinz restaur heinz die hard fan ketchup wish alway knew ketchup could without chemic aftertast found heinz sure give ketchup tri forget spice varieti purchas pack classic spice varieti first sir kensington experi definit way'

In [4]:
print(dataset['Text'][347])
print(dataset['p_text'][347])

I ordered these for my coffee themed wedding. When they arrived I had to fight off friends because they smelled and tasted so good. I literally had to hide the box to the wedding! They were a big hit on my wedding day and there were none left over that we put on the tables. It was a great tasting product for the lowest price I've seen!
order coffe theme wed arriv fight friend smell tast good liter hide box wed big hit wed day none left put tabl great tast product lowest price seen


In [5]:
from gensim.models import Word2Vec
sentences=[]
for sent in tqdm(p_text,position=0,leave=True):
    sentences.append(sent.split())

model=Word2Vec(sentences,min_count=5,size=100,workers=16)
w2v_words=dict(model.wv.vocab)
len(w2v_words)

100%|███████████████████████████████████████████████████████████████████████| 352744/352744 [00:03<00:00, 90395.23it/s]


21649

In [6]:
len(model.wv['good'])

100

In [7]:
text_vectors=[]
            
for sent in tqdm(sentences,position=0,leave=True):
    text_vec=np.zeros(100)
    word_cnt=0
    for word in sent:
        if word in w2v_words:
            text_vec+=model.wv[word]
            word_cnt+=1
            
    if word_cnt!=0:
        text_vec/=word_cnt
    text_vectors.append(text_vec)
    

    
    

100%|████████████████████████████████████████████████████████████████████████| 352744/352744 [01:53<00:00, 3101.78it/s]


In [8]:
text_vectors[0]

array([-5.14442891e-01, -1.42210147e-01, -3.26645710e-01, -2.44100249e-01,
        3.66878754e-01, -6.75766521e-01,  3.32401447e-01, -3.04388396e-03,
        9.10550290e-02,  3.52286058e-02,  3.72974381e-01,  4.74310299e-01,
       -6.15902463e-01,  1.81039186e-01,  2.08561194e-01,  2.11502080e-01,
       -9.25837468e-02,  1.83417681e-01, -3.36821617e-01,  2.03735961e-01,
       -9.08740363e-02, -4.06576183e-02,  1.63127245e-01,  1.66352998e-01,
        1.72943867e-02,  4.88694422e-02, -1.75357166e-01,  3.81489201e-02,
        3.67015706e-01,  2.04396301e-01, -5.24735602e-01,  6.44016188e-02,
       -1.06286212e-01, -7.72363675e-03,  2.62010783e-02,  4.34278399e-01,
       -2.36766308e-01, -4.62146314e-01,  1.90988092e-01,  1.75732060e-01,
        1.64253786e-01,  3.09478768e-01,  3.00733639e-01, -7.61912188e-01,
       -5.70509087e-01, -3.44225158e-01, -3.21389690e-01, -3.40706823e-01,
       -6.77278182e-02, -5.90705422e-03, -1.82634458e-01,  4.28602356e-02,
        6.83472390e-01, -

In [9]:
dataset['Score'].value_counts()

5    224921
4     48949
1     33856
3     26187
2     18831
Name: Score, dtype: int64

In [10]:
X=text_vectors
Y=[]
p_reviews=0
n_reviews=0
for score in dataset['Score'].values:
    if score >=4:
        Y.append(1)
        p_reviews+=1
    else:
        Y.append(0)
        n_reviews+=1
        
dataset['class']=Y
print(len(Y))
print(p_reviews)

352744
273870


In [11]:
from sklearn.neighbors import KNeighborsClassifier
neigh=KNeighborsClassifier(n_neighbors=25)
neigh.fit(X,Y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=25, p=2,
           weights='uniform')

In [ ]:
# K-Fold Cross Validation
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score,train_test_split

#x_train=X[0:7000]
#x_test=X[7000:] #70:30
#y_train=Y[0:7000]
#y_test=Y[7000:] #70:30
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=30)
optimal_k=1
a_max=0
print(x_train)
for k in tqdm(range(1,40,2),position=0,leave=True):
    knn=KNeighborsClassifier(n_neighbors=k)
    scores=cross_val_score(knn,x_train,y_train,cv=10,scoring='accuracy',n_jobs=-1)
    if scores.mean()>a_max:
        a_max=scores.mean()
        optimal_k=k
        
print(optimal_k)
knn=KNeighborsClassifier(n_neighbors=optimal_k)
knn.fit(x_train,y_train)
pred=knn.predict(x_test)

test_acc=accuracy_score(y_test,pred)*100
print(test_acc)

In [18]:
r="good product but can be better"
r=clean_html(r)
r=clean_punc(r)
test_review=[]
for w in r.split():
    if(w.isalpha() and len(w)>2 and w.lower() not in stop):
        w=(sno.stem(w.lower()))
        test_review.append(w)
        

vec=np.zeros(100)
cnt_words=0

for word in test_review:
    if word in w2v_words:
        vec+=model.wv[word]
        cnt_words+=1
        
if cnt_words!=0:
    vec/=cnt_words
    
print(test_review)
        
probs=neigh.predict_proba([vec])[0]
ans=""
if neigh.predict([vec])==1:
    ans+="Positive Review with "+str(probs[1])+" probabilty"
else:
    ans+="Negative Review with "+str(probs[0])+" probability"

print(ans)
    

['good', 'product', 'better']
Positive Review with 0.88 probabilty


In [20]:
from sklearn.externals import joblib 
  
# Save the model as a pickle in a file 
joblib.dump(neigh, 'amazon-review-analysis-knn.pkl') 

['amazon-review-analysis-knn.pkl']

In [21]:
joblib.dump(model, 'amazon-review-analysis-w2v.pkl') 

['amazon-review-analysis-w2v.pkl']